In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Optional
import random


In [ ]:
def sigmoid(z):
    """Сигмоидная функция активации"""
    return 1 / (1 + torch.exp(-z))

def train_neuron(features: List[List[float]], 
                 labels: List[int], 
                 initial_weights: List[float], 
                 initial_bias: float, 
                 learning_rate: float, 
                 epochs: int) -> Tuple[List[float], float, List[float]]:
    """
    Обучение одного нейрона с сигмоидной активацией используя градиентный спуск.
    
    Args:
        features: Список векторов признаков
        labels: Бинарные метки классов (0 или 1)
        initial_weights: Начальные веса
        initial_bias: Начальное смещение
        learning_rate: Скорость обучения
        epochs: Количество эпох
    
    Returns:
        Кортеж (обновленные веса, обновленное смещение, список NLL для каждой эпохи)
    """
    # Преобразуем в тензоры
    X = torch.tensor(features, dtype=torch.float32)
    y = torch.tensor(labels, dtype=torch.float32)
    weights = torch.tensor(initial_weights, dtype=torch.float32)
    bias = torch.tensor(initial_bias, dtype=torch.float32)
    
    n_samples = len(features)
    nll_values = []
    
    for epoch in range(epochs):
        # Прямой проход
        z = torch.matmul(X, weights) + bias  # линейная комбинация
        y_pred = sigmoid(z)  # применяем сигмоиду
        
        # Вычисляем Negative Log Likelihood
        # NLL = -1/n * sum(y*log(y_pred) + (1-y)*log(1-y_pred))
        epsilon = 1e-7  # для численной стабильности
        nll = -torch.mean(y * torch.log(y_pred + epsilon) + 
                         (1 - y) * torch.log(1 - y_pred + epsilon))
        nll_values.append(round(nll.item(), 4))
        
        # Вычисляем градиенты
        # dL/dy_pred = -y/y_pred + (1-y)/(1-y_pred)
        # dy_pred/dz = y_pred * (1 - y_pred) (производная сигмоиды)
        # dz/dw = X, dz/db = 1
        
        # Градиент по предсказаниям через chain rule
        error = y_pred - y  # упрощенная форма для NLL с сигмоидой
        
        # Градиенты по весам и смещению
        grad_weights = torch.matmul(X.T, error) / n_samples
        grad_bias = torch.mean(error)
        
        # Обновление параметров (градиентный спуск)
        weights = weights - learning_rate * grad_weights
        bias = bias - learning_rate * grad_bias
    
    return weights.tolist(), bias.item(), nll_values

In [3]:
def train_neuron_sgd(features: List[List[float]], 
                     labels: List[int], 
                     initial_weights: List[float], 
                     initial_bias: float, 
                     learning_rate: float, 
                     epochs: int) -> Tuple[List[float], float, List[float]]:
    """
    Стохастический градиентный спуск (SGD) - обновление на каждом примере.
    """
    X = torch.tensor(features, dtype=torch.float32)
    y = torch.tensor(labels, dtype=torch.float32)
    weights = torch.tensor(initial_weights, dtype=torch.float32)
    bias = torch.tensor(initial_bias, dtype=torch.float32)
    
    n_samples = len(features)
    nll_values = []
    
    for epoch in range(epochs):
        # Перемешиваем данные для каждой эпохи
        indices = list(range(n_samples))
        random.shuffle(indices)
        
        epoch_loss = 0.0
        
        for idx in indices:
            # Берем один пример
            x_i = X[idx]
            y_i = y[idx]
            
            # Прямой проход для одного примера
            z = torch.dot(x_i, weights) + bias
            y_pred = sigmoid(z)
            
            # Вычисляем loss для отслеживания
            epsilon = 1e-7
            loss = -(y_i * torch.log(y_pred + epsilon) + 
                    (1 - y_i) * torch.log(1 - y_pred + epsilon))
            epoch_loss += loss.item()
            
            # Градиент для одного примера
            error = y_pred - y_i
            
            # Обновление параметров сразу после каждого примера
            grad_weights = x_i * error
            grad_bias = error
            
            weights = weights - learning_rate * grad_weights
            bias = bias - learning_rate * grad_bias
        
        # Среднее NLL за эпоху
        avg_nll = epoch_loss / n_samples
        nll_values.append(round(avg_nll, 4))
    
    return weights.tolist(), bias.item(), nll_values

In [4]:
def train_neuron_mini_batch(features: List[List[float]], 
                            labels: List[int], 
                            initial_weights: List[float], 
                            initial_bias: float, 
                            learning_rate: float, 
                            epochs: int,
                            batch_size: int = 8) -> Tuple[List[float], float, List[float]]:
    """
    Mini-batch градиентный спуск - обновление на мини-батчах.
    """
    X = torch.tensor(features, dtype=torch.float32)
    y = torch.tensor(labels, dtype=torch.float32)
    weights = torch.tensor(initial_weights, dtype=torch.float32)
    bias = torch.tensor(initial_bias, dtype=torch.float32)
    
    n_samples = len(features)
    nll_values = []
    
    for epoch in range(epochs):
        # Перемешиваем данные
        indices = list(range(n_samples))
        random.shuffle(indices)
        
        epoch_loss = 0.0
        
        # Обрабатываем данные батчами
        for i in range(0, n_samples, batch_size):
            batch_indices = indices[i:min(i + batch_size, n_samples)]
            X_batch = X[batch_indices]
            y_batch = y[batch_indices]
            
            # Прямой проход для батча
            z = torch.matmul(X_batch, weights) + bias
            y_pred = sigmoid(z)
            
            # NLL для батча
            epsilon = 1e-7
            batch_loss = -torch.mean(y_batch * torch.log(y_pred + epsilon) + 
                                    (1 - y_batch) * torch.log(1 - y_pred + epsilon))
            epoch_loss += batch_loss.item() * len(batch_indices)
            
            # Градиенты для батча
            error = y_pred - y_batch
            grad_weights = torch.matmul(X_batch.T, error) / len(batch_indices)
            grad_bias = torch.mean(error)
            
            # Обновление параметров
            weights = weights - learning_rate * grad_weights
            bias = bias - learning_rate * grad_bias
        
        # Среднее NLL за эпоху
        avg_nll = epoch_loss / n_samples
        nll_values.append(round(avg_nll, 4))
    
    return weights.tolist(), bias.item(), nll_values

In [5]:
def generate_dataset(n_samples: int = 100, 
                     n_features: int = 2, 
                     seed: Optional[int] = None) -> Tuple[List[List[float]], List[int]]:
    """
    Генерация синтетического датасета для бинарной классификации.
    """
    if seed is not None:
        np.random.seed(seed)
        torch.manual_seed(seed)
    
    # Генерируем два кластера
    center1 = np.random.randn(n_features) * 2
    center2 = np.random.randn(n_features) * 2 + 3
    
    features = []
    labels = []
    
    for i in range(n_samples):
        if i < n_samples // 2:
            # Класс 0
            point = center1 + np.random.randn(n_features) * 0.5
            features.append(point.tolist())
            labels.append(0)
        else:
            # Класс 1
            point = center2 + np.random.randn(n_features) * 0.5
            features.append(point.tolist())
            labels.append(1)
    
    return features, labels

def visualize_convergence(methods_results: dict, title: str = "Сравнение методов оптимизации"):
    """
    Визуализация сходимости различных методов градиентного спуска.
    """
    plt.figure(figsize=(10, 6))
    
    for method_name, nll_values in methods_results.items():
        epochs = range(1, len(nll_values) + 1)
        plt.plot(epochs, nll_values, marker='o', label=method_name, linewidth=2)
    
    plt.xlabel('Эпоха', fontsize=12)
    plt.ylabel('NLL Loss', fontsize=12)
    plt.title(title, fontsize=14)
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def test_different_surfaces():
    """
    Тестирование на различных поверхностях потерь.
    """
    print("=" * 60)
    print("ТЕСТИРОВАНИЕ НА РАЗЛИЧНЫХ ПОВЕРХНОСТЯХ ПОТЕРЬ")
    print("=" * 60)
    
    # Параметры обучения
    learning_rate = 0.1
    epochs = 50
    
    # 1. Хорошо разделимые данные (простая поверхность)
    print("\n1. Хорошо разделимые данные (простая поверхность):")
    features1, labels1 = generate_dataset(100, 2, seed=42)
    initial_weights = [0.1, -0.2]
    initial_bias = 0.0
    
    results1 = {}
    
    # Классический градиентный спуск
    w, b, nll = train_neuron(features1, labels1, initial_weights, 
                             initial_bias, learning_rate, epochs)
    results1['Batch GD'] = nll
    print(f"Batch GD - Final NLL: {nll[-1]:.4f}")
    
    # SGD
    w, b, nll = train_neuron_sgd(features1, labels1, initial_weights, 
                                 initial_bias, learning_rate, epochs)
    results1['SGD'] = nll
    print(f"SGD - Final NLL: {nll[-1]:.4f}")
    
    # Mini-batch
    w, b, nll = train_neuron_mini_batch(features1, labels1, initial_weights, 
                                        initial_bias, learning_rate, epochs, batch_size=10)
    results1['Mini-batch (size=10)'] = nll
    print(f"Mini-batch - Final NLL: {nll[-1]:.4f}")
    
    # 2. Плохо разделимые данные (сложная поверхность)
    print("\n2. Плохо разделимые данные (сложная поверхность):")
    
    # Генерируем перекрывающиеся классы
    np.random.seed(123)
    features2 = []
    labels2 = []
    for i in range(100):
        x = np.random.randn(2) * 2
        # Нелинейная граница
        if x[0]**2 + x[1]**2 + np.random.randn() * 0.5 > 2:
            labels2.append(1)
        else:
            labels2.append(0)
        features2.append(x.tolist())
    
    results2 = {}
    
    # Классический градиентный спуск
    w, b, nll = train_neuron(features2, labels2, initial_weights, 
                             initial_bias, learning_rate * 0.5, epochs)
    results2['Batch GD'] = nll
    print(f"Batch GD - Final NLL: {nll[-1]:.4f}")
    
    # SGD
    w, b, nll = train_neuron_sgd(features2, labels2, initial_weights, 
                                 initial_bias, learning_rate * 0.5, epochs)
    results2['SGD'] = nll
    print(f"SGD - Final NLL: {nll[-1]:.4f}")
    
    # Mini-batch
    w, b, nll = train_neuron_mini_batch(features2, labels2, initial_weights, 
                                        initial_bias, learning_rate * 0.5, epochs, batch_size=10)
    results2['Mini-batch (size=10)'] = nll
    print(f"Mini-batch - Final NLL: {nll[-1]:.4f}")
    
    return results1, results2

In [6]:
print("БАЗОВЫЙ ПРИМЕР:")
print("-" * 40)
    
features = [[1.0, 2.0], [2.0, 1.0], [-1.0, -2.0]]
labels = [1, 0, 0]
initial_weights = [0.1, -0.2]
initial_bias = 0.0
learning_rate = 0.1
epochs = 2
    
updated_weights, updated_bias, nll_values = train_neuron(
    features, labels, initial_weights, initial_bias, learning_rate, epochs
)
    
print(f"Updated weights: {updated_weights}")
print(f"Updated bias: {updated_bias:.4f}")
print(f"NLL values: {nll_values}")
    
# Тест на большем датасете
print("\n" + "=" * 60)
print("ТЕСТ НА СИНТЕТИЧЕСКОМ ДАТАСЕТЕ (100 примеров):")
print("=" * 60)
    
# Генерируем датасет
features_large, labels_large = generate_dataset(100, 2, seed=42)
initial_weights_large = [0.0, 0.0]
initial_bias_large = 0.0
learning_rate_large = 0.5
epochs_large = 30
    
# Сравнение методов
methods_results = {}
    
print("\n1. Классический градиентный спуск (Batch GD):")
w1, b1, nll1 = train_neuron(features_large, labels_large, 
                                initial_weights_large, initial_bias_large, 
                                learning_rate_large, epochs_large)
methods_results['Batch GD'] = nll1
print(f"   Final weights: {[round(w, 4) for w in w1]}")
print(f"   Final bias: {b1:.4f}")
print(f"   Final NLL: {nll1[-1]:.4f}")
    
print("\n2. Стохастический градиентный спуск (SGD):")
w2, b2, nll2 = train_neuron_sgd(features_large, labels_large, 
                                    initial_weights_large, initial_bias_large, 
                                    learning_rate_large, epochs_large)
methods_results['SGD'] = nll2
print(f"   Final weights: {[round(w, 4) for w in w2]}")
print(f"   Final bias: {b2:.4f}")
print(f"   Final NLL: {nll2[-1]:.4f}")
    
print("\n3. Mini-batch градиентный спуск:")
w3, b3, nll3 = train_neuron_mini_batch(features_large, labels_large, 
                                           initial_weights_large, initial_bias_large, 
                                           learning_rate_large, epochs_large, 
                                           batch_size=10)
methods_results['Mini-batch (size=10)'] = nll3
print(f"   Final weights: {[round(w, 4) for w in w3]}")
print(f"   Final bias: {b3:.4f}")
print(f"   Final NLL: {nll3[-1]:.4f}")
    
# Тестирование на разных поверхностях
results_simple, results_complex = test_different_surfaces()
    
print("\n" + "=" * 60)
print("АНАЛИЗ РЕЗУЛЬТАТОВ:")
print("=" * 60)
print("""
    1. Batch GD (классический):
       - Стабильная сходимость
       - Использует все данные для каждого обновления
       - Может застревать в локальных минимумах
    
    2. SGD (стохастический):
       - Более шумная сходимость
       - Быстрее на больших датасетах
       - Может выскакивать из локальных минимумов
    
    3. Mini-batch:
       - Баланс между Batch GD и SGD
       - Хорошая скорость и стабильность
       - Оптимален для большинства задач
    
    Влияние формы поверхности:
    - На простых поверхностях все методы сходятся хорошо
    - На сложных поверхностях SGD может показывать лучшие результаты
    - Mini-batch обычно дает лучший баланс скорости и качества
""")

БАЗОВЫЙ ПРИМЕР:
----------------------------------------
Updated weights: [0.10698875039815903, -0.08469319343566895]
Updated bias: -0.0335
NLL values: [0.8006, 0.7631]

ТЕСТ НА СИНТЕТИЧЕСКОМ ДАТАСЕТЕ (100 примеров):

1. Классический градиентный спуск (Batch GD):
   Final weights: [-0.5984, 1.4607]
   Final bias: -1.4307
   Final NLL: 0.0570

2. Стохастический градиентный спуск (SGD):
   Final weights: [-1.5463, 3.5938]
   Final bias: -5.3089
   Final NLL: 0.0008

3. Mini-batch градиентный спуск:
   Final weights: [-1.1499, 2.5261]
   Final bias: -3.1924
   Final NLL: 0.0072
ТЕСТИРОВАНИЕ НА РАЗЛИЧНЫХ ПОВЕРХНОСТЯХ ПОТЕРЬ

1. Хорошо разделимые данные (простая поверхность):
Batch GD - Final NLL: 0.1489
SGD - Final NLL: 0.0023
Mini-batch - Final NLL: 0.0197

2. Плохо разделимые данные (сложная поверхность):
Batch GD - Final NLL: 0.5735
SGD - Final NLL: 0.5334
Mini-batch - Final NLL: 0.5238

АНАЛИЗ РЕЗУЛЬТАТОВ:

    1. Batch GD (классический):
       - Стабильная сходимость
       - Использ